# Additional data processing


This notebook corrects the monthly lake files to ensure that the values are always postive compared to the corresponding V20 monthly values.

In [3]:
import xarray as xr
# Load the V20 file
V20_cl = '/network/group/aopp/predict/TIP016_PAXTON_RPSPEEDY/ML4L/ECMWF_files/raw/climate.v020/climate.v020/639l_2/clake'
ds_V20cl = xr.open_dataset(V20_cl,engine='cfgrib',backend_kwargs={'indexpath': ''})

In [4]:
#Get all the lake files
import glob
root = '/network/group/aopp/predict/TIP016_PAXTON_RPSPEEDY/ML4L/ECMWF_files/raw/' 
lake_files = sorted(glob.glob(f'{root}BonusClimate/clake*'))

In [113]:
def correct_monthly_lake_file(f):
    
    print(f)
    #Load the monthly lake file
    ds_lake = xr.open_dataset(f,engine='cfgrib',backend_kwargs={'indexpath': ''})
    
    #Get the difference
    delta = ds_lake - ds_V20cl
    v = delta.cl
    print("Number of negative values = ",len(v[np.where(v<0)]))
  

    #Extract the data
    monthly_cl_array = delta.cl.data
    V20_cl_array = ds_V20cl.cl.data
    
    
    #Anything less than zero is now zero
    new_values = monthly_cl_array.clip(min=0) #
    
    #And then add on the static V20 field
    new_values += V20_cl_array
    ds_lake.cl.data = new_values

    
    new_delta = ds_lake - ds_V20cl
    v = new_delta.cl.data
    print("Number of negative values = ",len(v[np.where(v<0)]))
    assert (len(v[np.where(v<0)])== 0.0)
    
    
    display(ds_lake)


In [115]:
import sys
for f in lake_files:
    correct_monthly_lake_file(f)
    
    sys.exit()


/network/group/aopp/predict/TIP016_PAXTON_RPSPEEDY/ML4L/ECMWF_files/raw/BonusClimate/clake_639l2_month01
Number of negative values =  33912
Number of negative values =  0


<xarray.Dataset>
Dimensions:     (values: 542080)
Coordinates:
    time        datetime64[ns] ...
    step        timedelta64[ns] ...
    surface     float64 ...
    latitude    (values) float64 ...
    longitude   (values) float64 ...
    valid_time  datetime64[ns] ...
Dimensions without coordinates: values
Data variables:
    cl          (values) float32 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2022-09-07T10:37 GRIB to CDM+CF via cfgrib-0.9.1...

SystemExit: 

/home/k/kimpson/bin/conda/envs/workhorse/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [138]:
import pandas as pd
path = '/network/group/aopp/predict/TIP016_PAXTON_RPSPEEDY/ML4L/ECMWF_files/raw/processed_data/joined_data/'
f = '2016_RML.parquet'
df = pd.read_parquet(path+f).reset_index()

#Pass monthly clake as a v20 correction
#df['clake_monthly_value'] = df['clake_monthly_value'] - df['cl_v20'] 
#df['clake_monthly_value'] =  df['clake_monthly_value'].clip(lower=0)
assert (df['clake_monthly_value'] >= 0).all() # the monthly cl corrections should always be positive



AssertionError: 

In [154]:
f = '/network/group/aopp/predict/TIP016_PAXTON_RPSPEEDY/ML4L/ECMWF_files/raw/processed_data/joined_data/2016_SEPTEMBER.parquet'

In [155]:
df = pd.read_parquet(f).reset_index()


In [160]:
df.columns

Index(['index', 'sp', 'msl', 'u10', 'v10', 't2m', 'aluvp', 'aluvd', 'alnip',
       'alnid', 'istl1', 'istl2', 'sd', 'd2m', 'fal', 'skt', 'slt_v15',
       'sdfor_v15', 'sdor_v15', 'cvl_v15', 'lsm_v15', 'isor_v15', 'tvl_v15',
       'tvh_v15', 'cvh_v15', 'si10_v15', 'anor_v15', 'cl_v15', 'dl_v15',
       'z_v15', 'slor_v15', 'sdor_v20', 'cvl_v20', 'lsm_v20', 'isor_v20',
       'cvh_v20', 'si10_v20', 'anor_v20', 'cl_v20', 'dl_v20', 'z_v20',
       'slor_v20', 'clake_monthly_value', 'cl_saline', 'MODIS_LST',
       'latitude_ERA', 'longitude_ERA', 'time', 'number_of_modis_observations',
       'skt_unnormalised'],
      dtype='object')

In [161]:
cl_perm = df['cl_v15']

In [156]:
cl = df.clake_monthly_value.min()

In [157]:
cl

-0.1957032

In [163]:
diff = cl_perm - cl

In [165]:
diff.max()

11.527483

In [149]:
assert (df['clake_monthly_value'] >= 0).all() # the monthly cl corrections should always be positive

AssertionError: 

In [140]:
cl = df.clake_monthly_value

In [142]:
df

,index,sp,msl,u10,v10,t2m,aluvp,aluvd,alnip,alnid,...,slor_v20,clake_monthly_value,cl_saline,cl_saline_max,MODIS_LST,latitude_ERA,longitude_ERA,time,number_of_modis_observations,skt_unnormalised
0,0,0.319781,-3.312112,-0.272708,-0.433632,-1.281853,-0.540116,-0.530686,-1.221462,-1.287657,...,-0.876279,-0.264192,-0.069769,-0.069769,268.034760,54.660397,-163.59375,2016-01-01 00:00:00,3,267.247559
1,1,0.488203,-3.344310,-1.563556,-0.321791,-0.938681,-0.405089,-0.468140,-1.542488,-1.693358,...,-0.013794,-0.263227,-0.069769,-0.069769,272.575195,54.660397,-163.12500,2016-01-01 00:00:00,5,277.714355
2,2,0.356893,-3.287817,-0.523570,0.267331,-1.176200,-0.585751,-0.564334,-0.857613,-0.884000,...,-0.114521,-0.264192,-0.069769,-0.069769,264.893341,54.941427,-163.59375,2016-01-01 00:00:00,1,270.976074
3,3,0.468569,-3.296013,-1.536535,0.212600,-0.953725,-0.405089,-0.468140,-1.542488,-1.693358,...,-0.013794,-0.210984,-0.069769,-0.069769,269.294556,54.941427,-163.12500,2016-01-01 00:00:00,1,277.122559
4,4,0.563676,-3.255912,-1.270628,0.163648,-0.969454,-0.405089,-0.468140,-1.542488,-1.693358,...,-0.013794,-0.264192,-0.069769,-0.069769,272.096771,55.222457,-163.12500,2016-01-01 00:00:00,13,277.179199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37481436,3011763,0.996274,1.173116,2.265275,0.176417,-1.829187,-0.405095,-0.468147,-1.542491,-1.693358,...,-0.013794,-0.264192,-0.069769,-0.069769,258.804779,70.117048,-145.50000,2016-12-31 23:00:00,14,260.710632
37481437,3011764,0.985719,1.173702,2.505697,-0.401147,-1.835227,-0.405095,-0.468147,-1.542491,-1.693358,...,-0.013794,-0.261306,-0.069769,-0.069769,258.174957,70.117048,-144.75000,2016-12-31 23:00:00,5,260.425476
37481438,3011765,0.931810,1.176921,2.693306,-0.782223,-1.882982,-0.405095,-0.468147,-1.542491,-1.693358,...,-0.013794,-0.214426,-0.069769,-0.069769,257.600525,70.117048,-144.00000,2016-12-31 23:00:00,22,259.671570
37481439,3011766,0.931016,1.164042,2.973031,-1.040240,-1.899964,-0.405095,-0.468147,-1.542491,-1.693358,...,-0.013794,-0.252729,-0.069769,-0.069769,258.027435,70.117048,-143.25000,2016-12-31 23:00:00,16,259.351257


In [131]:
all_monthly_files = sorted(glob.glob(path+'RapidsHaversine_MODIS_ERA_*'))

In [166]:
1+1

2

In [132]:
all_monthly_files

['/network/group/aopp/predict/TIP016_PAXTON_RPSPEEDY/ML4L/ECMWF_files/raw/processed_data/joined_data/RapidsHaversine_MODIS_ERA_2015_01.parquet',
 '/network/group/aopp/predict/TIP016_PAXTON_RPSPEEDY/ML4L/ECMWF_files/raw/processed_data/joined_data/RapidsHaversine_MODIS_ERA_2015_02.parquet',
 '/network/group/aopp/predict/TIP016_PAXTON_RPSPEEDY/ML4L/ECMWF_files/raw/processed_data/joined_data/RapidsHaversine_MODIS_ERA_2015_03.parquet',
 '/network/group/aopp/predict/TIP016_PAXTON_RPSPEEDY/ML4L/ECMWF_files/raw/processed_data/joined_data/RapidsHaversine_MODIS_ERA_2015_04.parquet',
 '/network/group/aopp/predict/TIP016_PAXTON_RPSPEEDY/ML4L/ECMWF_files/raw/processed_data/joined_data/RapidsHaversine_MODIS_ERA_2015_05.parquet',
 '/network/group/aopp/predict/TIP016_PAXTON_RPSPEEDY/ML4L/ECMWF_files/raw/processed_data/joined_data/RapidsHaversine_MODIS_ERA_2015_06.parquet',
 '/network/group/aopp/predict/TIP016_PAXTON_RPSPEEDY/ML4L/ECMWF_files/raw/processed_data/joined_data/RapidsHaversine_MODIS_ERA_201

In [145]:
training_years =   ['2016']

In [144]:
years = np.array_split(all_monthly_files, len(all_monthly_files)/12)




def extract_year_from_filename(f):
    
    print(f.split('_')[-2])




for months in years:
    extract_year_from_filename(months[0])
    sys.exit()

2015


SystemExit: 

In [130]:
df.query('clake_monthly_value < 0.0')

,latitude_ERA,longitude_ERA,latitude_MODIS,longitude_MODIS,MODIS_LST,u10,v10,sp,msl,t2m,...,cl_v20,dl_v20,z_v20,slor_v20,clake_monthly_value,cl_saline,heightAboveGround,H_distance_km,number_of_modis_observations,time


In [128]:
df['clake_monthly_value'] > 0

0           True
1           True
2           True
3           True
4           True
           ...  
3035259    False
3035260     True
3035261     True
3035262     True
3035263    False
Name: clake_monthly_value, Length: 3035264, dtype: bool

In [127]:
df['clake_monthly_value']

0          0.001129
1          0.001358
2          0.002233
3          0.003566
4          0.000443
             ...   
3035259    0.000000
3035260    0.000038
3035261    0.000652
3035262    0.000150
3035263    0.000000
Name: clake_monthly_value, Length: 3035264, dtype: float32

In [8]:
f = '/network/group/aopp/predict/TIP016_PAXTON_RPSPEEDY/ML4L/ECMWF_files/raw/BonusClimate/clake_639l2_month01'
ds_lake = xr.open_dataset(f,engine='cfgrib',backend_kwargs={'indexpath': ''})


In [116]:
ds_lake

<xarray.Dataset>
Dimensions:     (values: 542080)
Coordinates:
    time        datetime64[ns] ...
    step        timedelta64[ns] ...
    surface     float64 ...
    latitude    (values) float64 ...
    longitude   (values) float64 ...
    valid_time  datetime64[ns] ...
Dimensions without coordinates: values
Data variables:
    cl          (values) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2022-09-07T08:24 GRIB to CDM+CF via cfgrib-0.9.1...

In [9]:
delta = ds_lake-ds_V20cl

In [21]:
ds = delta.cl

In [47]:
vals = ds.data

In [44]:
vals = vals.clip(min=0)


In [48]:
np.where(vals<0)

(array([  1954,   1958,   2111, ..., 533411, 533412, 534997]),)

In [39]:
import numpy as np
vals = vals[np.where(vals<0)] = 0

TypeError: 'int' object does not support item assignment

In [33]:
ds.data = vals

In [34]:
ds

<xarray.DataArray 'cl' (values: 542080)>
array([1., 0., 0., ..., 0., 0., 0.], dtype=float32)
Coordinates:
    time        datetime64[ns] 2011-01-31
    step        timedelta64[ns] 00:00:00
    surface     float64 0.0
    latitude    (values) float64 89.78 89.78 89.78 ... -89.78 -89.78 -89.78
    longitude   (values) float64 0.0 20.0 40.0 60.0 ... 280.0 300.0 320.0 340.0
    valid_time  datetime64[ns] 2011-01-31
Dimensions without coordinates: values

In [24]:
ds.where(lambda x: x>0.8)

<xarray.DataArray 'cl' (values: 542080)>
array([nan, nan, nan, ..., nan, nan, nan], dtype=float32)
Coordinates:
    time        datetime64[ns] 2011-01-31
    step        timedelta64[ns] 00:00:00
    surface     float64 0.0
    latitude    (values) float64 89.78 89.78 89.78 ... -89.78 -89.78 -89.78
    longitude   (values) float64 0.0 20.0 40.0 60.0 ... 280.0 300.0 320.0 340.0
    valid_time  datetime64[ns] 2011-01-31
Dimensions without coordinates: values

In [20]:
ds.where(ds.cl=0.0)

SyntaxError: keyword can't be an expression (1987568510.py, line 1)

In [15]:
df = delta.to_dataframe().reset_index()

In [16]:
df

,values,time,step,surface,latitude,longitude,valid_time,cl
0,0,2011-01-31,0 days,0.0,89.784877,0.0,2011-01-31,0.0
1,1,2011-01-31,0 days,0.0,89.784877,20.0,2011-01-31,0.0
2,2,2011-01-31,0 days,0.0,89.784877,40.0,2011-01-31,0.0
3,3,2011-01-31,0 days,0.0,89.784877,60.0,2011-01-31,0.0
4,4,2011-01-31,0 days,0.0,89.784877,80.0,2011-01-31,0.0
...,...,...,...,...,...,...,...,...
542075,542075,2011-01-31,0 days,0.0,-89.784877,260.0,2011-01-31,0.0
542076,542076,2011-01-31,0 days,0.0,-89.784877,280.0,2011-01-31,0.0
542077,542077,2011-01-31,0 days,0.0,-89.784877,300.0,2011-01-31,0.0
542078,542078,2011-01-31,0 days,0.0,-89.784877,320.0,2011-01-31,0.0
